## Import Libraries

In [1]:
import json, os, glob, tqdm
import pandas as pd
import numpy as np

import geopandas as gpd

## Import Json Metadata

In [2]:
final_path = "Data\\Restaurant_Spatial-Analytical.json"
with open(final_path, 'r') as f:
    data = json.load(f)

print("Total : ",len(data['result']))

Total :  58376


In [3]:
columns = ['geo_id', 'name' ,'lat', 'lng', 'division', 'active_status', 'types', 'address', 'vicinity', 'rating', 'user_ratings_total', 'facility_cnt', 'contact', 'current_time', 'current_date',
           'opening_hours', 'hours_stats', 'web', 'flink', 'plink', 'g_map_url']
general_df = pd.DataFrame(columns=columns)
i = 0
for rec in tqdm.tqdm(data['result']):
    tmp = {'geo_id': rec['geo_id'], 'name':rec['name'] ,'lat':rec['geometry']['location']['lat'], 'lng':rec['geometry']['location']['lng'],'division':rec['division'], 'active_status':rec['active_status'], 
           'types':rec['types'][0], 'address':rec['address'], 'vicinity':rec['vicinity'],'rating':rec['rating'],'user_ratings_total':rec['user_ratings_total'],
           'facility_cnt':len(rec['facility'].keys()),'contact':rec['contact']['phone'],'current_time':rec['current_time'],'current_date':rec['current_date'],
           'opening_hours':"No" if not rec['opening_hours'].keys() else "Yes",'hours_stats':"No" if not rec['hours_stats'].keys() else "Yes", 'web' : rec['other_url']['web'], 
           'flink':rec['other_url']['flink'], 'plink':rec['other_url']['plink'],'g_map_url':rec['g_map_url']}
       #     'lng':rec['geometry'],'lng':rec['geometry']}
    # print(tmp)
    general_df.loc[len(general_df)] = tmp

    # if i==9 : break
    i = i + 1

100%|██████████| 58376/58376 [10:23<00:00, 93.59it/s] 


In [4]:
general_df.tail()

,geo_id,name,lat,lng,division,active_status,types,address,vicinity,rating,...,facility_cnt,contact,current_time,current_date,opening_hours,hours_stats,web,flink,plink,g_map_url
58371,24.649161591.6368225,তৃপ্তি স্ন্যাক্স এন্ড সুইটস্,24.649162,91.636822,Sylhet,Permanently closed,Restaurant,"JJXP+MP8, Sherpur - Padulla Rd",None,1.0,...,0,None,12:56:30 PM,2024-11-24,No,No,No,No,No,https://www.google.com/maps/place/%E0%A6%A4%E0...
58372,25.285854491.7205566,Pine leaf restaurant,25.285854,91.720557,India,Yes,Family restaurant,"Khliehshnong, Cherrapunji, Meghalaya 793108, I...","7PPC+86 Cherrapunji, Meghalaya, India",3.8,...,8,+91 76300 93100,12:56:48 PM,2024-11-24,No,No,No,No,No,https://www.google.com/maps/place/Pine+leaf+re...
58373,24.813389491.7559229,Bismillah Tanduri Restaurant,24.813389,91.755923,Sylhet,Yes,Restaurant,"RQ74+993 Bus Station, Biswanath Thana Road, Bi...",None,3.2,...,7,01712-148079,12:57:04 PM,2024-11-24,Yes,Yes,No,No,No,https://www.google.com/maps/place/Bismillah+Ta...
58374,24.670919491.6699958,তালিমুস সুন্নাহ বালিকা মাদ্রাসা,24.670919,91.669996,Sylhet,Yes,Madrilian restaurant,"MMC9+9X9, Rahmatpur",None,,...,5,None,12:57:17 PM,2024-11-24,No,No,No,No,No,https://www.google.com/maps/place/%E0%A6%A4%E0...
58375,24.754106591.5949437,Rumel restaurant,24.754106,91.594944,Sylhet,Yes,Restaurant,"QH3V+MX2, Road, Syedpur",None,,...,8,01745-671928,12:57:30 PM,2024-11-24,Yes,No,No,No,No,https://www.google.com/maps/place/Rumel+restau...


In [5]:
data_df = pd.json_normalize(data, record_path='result', meta=None)

## Key Features of Restaurant Data

In [6]:
ficility_df = data_df[['geo_id','facility.Accessibility','facility.Service options', 'facility.Highlights', 'facility.Offerings', 'facility.Dining options',
       'facility.Amenities', 'facility.Atmosphere', 'facility.Crowd','facility.Planning', 'facility.Children', 'facility.Parking','facility.Payments', 
       'facility.Pets', 'facility.From the business','facility.Recycling', 'opening_hours.special_day','facility.Popular for', 'facility.Activities']]

ficility_df.columns = ficility_df.columns.str.replace('facility.', '', regex=False)
ficility_df.columns = ficility_df.columns.str.replace('opening_hours.', '', regex=False)
ficility_df = ficility_df.applymap(lambda x: ','.join(x) if isinstance(x, list) else x)
ficility_df.tail()

C:\Users\USER\AppData\Local\Temp\ipykernel_115684\3485267132.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ficility_df = ficility_df.applymap(lambda x: ','.join(x) if isinstance(x, list) else x)


,geo_id,Accessibility,Service options,Highlights,Offerings,Dining options,Amenities,Atmosphere,Crowd,Planning,Children,Parking,Payments,Pets,From the business,Recycling,special_day,Popular for,Activities
58371,24.649161591.6368225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58372,25.285854491.7205566,NaN,"Kerbside pickup,Takeaway,Dine-in",NaN,"All you can eat,Coffee,Halal food,Happy-hour f...","Breakfast,Lunch,Dinner,Dessert",Toilets,Casual,"Family friendly,Groups,LGBTQ+ friendly",NaN,Good for kids,"Free of charge street parking,Free parking lot...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
58373,24.813389491.7559229,NaN,"Delivery,Takeaway,Dine-in",NaN,"Halal food,Small plates","Lunch,Dinner",NaN,Casual,Groups,NaN,Good for kids,Free of charge street parking,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58374,24.670919491.6699958,NaN,NaN,NaN,"Halal food,Small plates",NaN,NaN,Casual,Groups,NaN,Good for kids,"Free of charge street parking,Free parking lot",NaN,NaN,NaN,NaN,NaN,NaN,NaN
58375,24.754106591.5949437,NaN,"Takeaway,Dine-in",NaN,"Halal food,Small plates","Lunch,Dinner",Toilets,Casual,Groups,NaN,Good for kids,Free of charge street parking,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Hourly stat of the Restaurant Data

In [7]:
hour_stats_df = data_df[['geo_id','opening_hours.Thursday.open', 'opening_hours.Thursday.close',
       'opening_hours.Friday.open', 'opening_hours.Friday.close',
       'opening_hours.Saturday.open', 'opening_hours.Saturday.close',
       'opening_hours.Sunday.open', 'opening_hours.Sunday.close',
       'opening_hours.Monday.open', 'opening_hours.Monday.close',
       'opening_hours.Tuesday.open', 'opening_hours.Tuesday.close',
       'opening_hours.Wednesday.open', 'opening_hours.Wednesday.close', 
        'hours_stats.Sunday.12 AM', 'hours_stats.Sunday.1 AM',
       'hours_stats.Sunday.2 AM', 'hours_stats.Sunday.3 AM',
       'hours_stats.Sunday.4 AM', 'hours_stats.Sunday.5 AM',
       'hours_stats.Sunday.6 AM', 'hours_stats.Sunday.7 AM',
       'hours_stats.Sunday.8 AM', 'hours_stats.Sunday.9 AM',
       'hours_stats.Sunday.10 AM', 'hours_stats.Sunday.11 AM',
       'hours_stats.Sunday.12 PM', 'hours_stats.Sunday.1 PM',
       'hours_stats.Sunday.2 PM', 'hours_stats.Sunday.3 PM',
       'hours_stats.Sunday.4 PM', 'hours_stats.Sunday.5 PM',
       'hours_stats.Sunday.6 PM', 'hours_stats.Sunday.7 PM',
       'hours_stats.Sunday.8 PM', 'hours_stats.Sunday.9 PM',
       'hours_stats.Sunday.10 PM', 'hours_stats.Sunday.11 PM',
       'hours_stats.Monday.12 AM', 'hours_stats.Monday.1 AM',
       'hours_stats.Monday.2 AM', 'hours_stats.Monday.3 AM',
       'hours_stats.Monday.4 AM', 'hours_stats.Monday.5 AM',
       'hours_stats.Monday.6 AM', 'hours_stats.Monday.7 AM',
       'hours_stats.Monday.8 AM', 'hours_stats.Monday.9 AM',
       'hours_stats.Monday.10 AM', 'hours_stats.Monday.11 AM',
       'hours_stats.Monday.12 PM', 'hours_stats.Monday.1 PM',
       'hours_stats.Monday.2 PM', 'hours_stats.Monday.3 PM',
       'hours_stats.Monday.4 PM', 'hours_stats.Monday.5 PM',
       'hours_stats.Monday.6 PM', 'hours_stats.Monday.7 PM',
       'hours_stats.Monday.8 PM', 'hours_stats.Monday.9 PM',
       'hours_stats.Monday.10 PM', 'hours_stats.Monday.11 PM',
        'hours_stats.Tuesday.12 AM', 'hours_stats.Tuesday.1 AM',
       'hours_stats.Tuesday.2 AM', 'hours_stats.Tuesday.3 AM',
       'hours_stats.Tuesday.4 AM', 'hours_stats.Tuesday.5 AM',
       'hours_stats.Tuesday.6 AM', 'hours_stats.Tuesday.7 AM',
       'hours_stats.Tuesday.8 AM', 'hours_stats.Tuesday.9 AM',
       'hours_stats.Tuesday.10 AM', 'hours_stats.Tuesday.11 AM',
       'hours_stats.Tuesday.12 PM', 'hours_stats.Tuesday.1 PM',
       'hours_stats.Tuesday.2 PM', 'hours_stats.Tuesday.3 PM',
       'hours_stats.Tuesday.4 PM', 'hours_stats.Tuesday.5 PM',
       'hours_stats.Tuesday.6 PM', 'hours_stats.Tuesday.7 PM',
       'hours_stats.Tuesday.8 PM', 'hours_stats.Tuesday.9 PM',
       'hours_stats.Tuesday.10 PM', 'hours_stats.Tuesday.11 PM',
        'hours_stats.Wednesday.12 AM', 'hours_stats.Wednesday.1 AM',
       'hours_stats.Wednesday.2 AM', 'hours_stats.Wednesday.3 AM',
       'hours_stats.Wednesday.4 AM', 'hours_stats.Wednesday.5 AM',
       'hours_stats.Wednesday.6 AM', 'hours_stats.Wednesday.7 AM',
       'hours_stats.Wednesday.8 AM', 'hours_stats.Wednesday.9 AM',
       'hours_stats.Wednesday.10 AM', 'hours_stats.Wednesday.11 AM',
       'hours_stats.Wednesday.12 PM', 'hours_stats.Wednesday.1 PM',
       'hours_stats.Wednesday.2 PM', 'hours_stats.Wednesday.3 PM',
       'hours_stats.Wednesday.4 PM', 'hours_stats.Wednesday.5 PM',
       'hours_stats.Wednesday.6 PM', 'hours_stats.Wednesday.7 PM',
       'hours_stats.Wednesday.8 PM', 'hours_stats.Wednesday.9 PM',
       'hours_stats.Wednesday.10 PM', 'hours_stats.Wednesday.11 PM',
        'hours_stats.Thrusday.12 AM', 'hours_stats.Thrusday.1 AM',
       'hours_stats.Thrusday.2 AM', 'hours_stats.Thrusday.3 AM',
       'hours_stats.Thrusday.4 AM', 'hours_stats.Thrusday.5 AM',
       'hours_stats.Thrusday.6 AM', 'hours_stats.Thrusday.7 AM',
       'hours_stats.Thrusday.8 AM', 'hours_stats.Thrusday.9 AM',
       'hours_stats.Thrusday.10 AM', 'hours_stats.Thrusday.11 AM',
       'hours_stats.Thrusday.12 PM', 'hours_stats.Thrusday.1 PM',
       'hours_stats.Thrusday.2 PM', 'hours_stats.Thrusday.3 PM',
       'hours_stats.Thrusday.4 PM', 'hours_stats.Thrusday.5 PM',
       'hours_stats.Thrusday.6 PM', 'hours_stats.Thrusday.7 PM',
       'hours_stats.Thrusday.8 PM', 'hours_stats.Thrusday.9 PM',
       'hours_stats.Thrusday.10 PM', 'hours_stats.Thrusday.11 PM',
       'hours_stats.Friday.12 AM', 'hours_stats.Friday.1 AM',
       'hours_stats.Friday.2 AM', 'hours_stats.Friday.3 AM',
       'hours_stats.Friday.4 AM', 'hours_stats.Friday.5 AM',
       'hours_stats.Friday.6 AM', 'hours_stats.Friday.7 AM',
       'hours_stats.Friday.8 AM', 'hours_stats.Friday.9 AM',
       'hours_stats.Friday.10 AM', 'hours_stats.Friday.11 AM',
       'hours_stats.Friday.12 PM', 'hours_stats.Friday.1 PM',
       'hours_stats.Friday.2 PM', 'hours_stats.Friday.3 PM',
       'hours_stats.Friday.4 PM', 'hours_stats.Friday.5 PM',
       'hours_stats.Friday.6 PM', 'hours_stats.Friday.7 PM',
       'hours_stats.Friday.8 PM', 'hours_stats.Friday.9 PM',
       'hours_stats.Friday.10 PM', 'hours_stats.Friday.11 PM',
        'hours_stats.Saturday.12 AM', 'hours_stats.Saturday.1 AM',
       'hours_stats.Saturday.2 AM', 'hours_stats.Saturday.3 AM',
       'hours_stats.Saturday.4 AM', 'hours_stats.Saturday.5 AM',
       'hours_stats.Saturday.6 AM', 'hours_stats.Saturday.7 AM',
       'hours_stats.Saturday.8 AM', 'hours_stats.Saturday.9 AM',
       'hours_stats.Saturday.10 AM', 'hours_stats.Saturday.11 AM',
       'hours_stats.Saturday.12 PM', 'hours_stats.Saturday.1 PM',
       'hours_stats.Saturday.2 PM', 'hours_stats.Saturday.3 PM',
       'hours_stats.Saturday.4 PM', 'hours_stats.Saturday.5 PM',
       'hours_stats.Saturday.6 PM', 'hours_stats.Saturday.7 PM',
       'hours_stats.Saturday.8 PM', 'hours_stats.Saturday.9 PM',
       'hours_stats.Saturday.10 PM', 'hours_stats.Saturday.11 PM']]

hour_stats_df.columns = hour_stats_df.columns.str.replace('hours_stats.', '', regex=False)
hour_stats_df.columns = hour_stats_df.columns.str.replace('opening_hours.', '', regex=False)
hour_stats_df.tail()

,geo_id,Thursday.open,Thursday.close,Friday.open,Friday.close,Saturday.open,Saturday.close,Sunday.open,Sunday.close,Monday.open,...,Saturday.2 PM,Saturday.3 PM,Saturday.4 PM,Saturday.5 PM,Saturday.6 PM,Saturday.7 PM,Saturday.8 PM,Saturday.9 PM,Saturday.10 PM,Saturday.11 PM
58371,24.649161591.6368225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58372,25.285854491.7205566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58373,24.813389491.7559229,9 AM,12 AM,9 AM,12 AM,9 AM,12 AM,9 AM,12 AM,9 AM,...,69.0,61.0,100.0,46.0,30.0,15.0,38.0,15.0,7.0,0.0
58374,24.670919491.6699958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58375,24.754106591.5949437,6:29,11:30 AM,6:29,11:30 AM,6:29,11:30 AM,6:29,11:30 AM,6:29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merged all Columns in Single Data Frame

In [8]:
merged_df = pd.merge(ficility_df, hour_stats_df, on='geo_id', how='outer')  # First merge df1 and df2
merged_df = pd.merge(merged_df, general_df, on='geo_id', how='outer')  # Then merge with df3

merged_df.tail()

,geo_id,Accessibility,Service options,Highlights,Offerings,Dining options,Amenities,Atmosphere,Crowd,Planning,...,facility_cnt,contact,current_time,current_date,opening_hours,hours_stats,web,flink,plink,g_map_url
58371,26.997987988.1668416,NaN,NaN,NaN,Dishes,NaN,NaN,NaN,NaN,NaN,...,1,None,02:03:28 AM,2024-11-22,No,No,No,No,No,https://www.google.com/maps/place/Narmaya+Rai+...
58372,27.065092488.466683,NaN,"Takeaway,Dine-in",NaN,Dishes,NaN,NaN,Casual,Groups,NaN,...,5,None,02:26:34 AM,2024-11-22,No,No,No,No,No,https://www.google.com/maps/place/Arya+Fast+Fo...
58373,27.066018788.4684024,NaN,Dine-in,NaN,Dishes,NaN,NaN,NaN,NaN,NaN,...,2,None,01:18:14 AM,2024-11-22,No,No,No,No,No,https://www.google.com/maps/place/Yummy+Fast+F...
58374,27.325659888.6118066,"Wheelchair-accessible car park,Wheelchair-acce...","No-contact delivery,Delivery,Takeaway,Dine-in",NaN,"Coffee,Halal food,Small plates,Cuisine,Dishes","Breakfast,Brunch,Lunch,Dinner,Dessert","Gender-neutral toilets,Toilets,Wi-Fi,Free Wi-Fi","Casual,Cosy","Family friendly,Groups",Accepts reservations,...,10,+91 99324 80018,01:43:45 AM,2024-11-22,Yes,Yes,https://www.swiggy.com/restaurants/khan-uncle-...,No,No,https://www.google.com/maps/place/Khan+Uncle%2...
58375,27.348644968.5347163,"Wheelchair-accessible car park,Wheelchair-acce...","Outdoor seating,Kerbside pickup,No-contact del...",Fireplace,"All you can eat,Coffee,Halal food,Late-night f...","Brunch,Lunch,Dinner,Dessert,Seating",Toilets,"Casual,Cosy",Groups,Accepts reservations,...,13,None,07:21:08 PM,2024-11-21,Yes,Yes,No,No,No,https://www.google.com/maps/place/Fast+foodies...
